<div style="font-size: 200%; font-weight: bold; color: maroon;">Pandas UDF 2 - 
<span style="color: green;">operaciones sobre grupos</span></div>
<!-- Paulo Villegas, v. 1.1, 2022 -->

Usamos ahora Pandas UDFs para operaciones algo más complejas. Vamos a volver a usar el dataset del censo

In [ ]:
spark

# 1. Carga de datos

In [ ]:
# Read the data
censo = spark.read.format('parquet').load('data/censo-join.parquet')

censo = censo.repartition(8)

In [ ]:
censo.printSchema()

## 1.1 Emigración

Volvemos a calcular las personas que no viven en su provincia de nacimiento

In [ ]:
emigrantes = censo.filter( censo.CPRO != censo.CPRON )

In [ ]:
cem = emigrantes.count()

In [ ]:
cem/censo.count()

# 2. Pandas UDF: Grouped Map

En este apartado usaremos el [API de funciones Pandas](https://spark.apache.org/docs/latest/api/python/user_guide/arrow_pandas.html#pandas-function-apis) para UDFs, aplicándolo a grupos

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

Queremos calcular, para cada emigrante, 
 * cuál es la edad en la que la persona llega al municipio, 
 * comparada con la *edad media* en que llegan los emigrantes a esa provincia

In [ ]:
def age_diff_udf(dfprov: pd.DataFrame) -> pd.DataFrame:
    '''
    Para cada fila del DataFrame, calcula la edad de llegada al municipio de la persona de esa fila, y 
    obtiene la diferencia de esa edad con la edad media de llegada de todas las personas que han emigrado 
    al municipio
    '''
    # La edad a la llegada la calculamos como el año de llegada al municipio menos el año de nacimiento
    # Como el esquema del DataFrame de Spark es plano (casi todos los campos son strings), tenemos que pasar
    # las series de Pandas a enteros para poder restar
    edad_llegada = dfprov.ANOM.astype(int) - dfprov.ANAC.astype(int)
    
    # Añadimos ahora al DataFrame el campo de salida: la edad relativa a la media de edades de llegada
    dfprov['edad_rel'] = edad_llegada - edad_llegada.mean()
    
    # Nuestro DataFrame de salida contiene el campo generado "edad_rel", más unas columnas adicionales
    return dfprov[['edad_rel', 'ANAC', 'ANOM', 'NOMBRE_PRO']]


# Define el esquema del DataFrame de Pandas devuelto por la UDF
age_diff_schema = "edad_rel int, ANAC string, ANOM string, NOMBRE_PRO string"

In [ ]:
# Aplicamos el cálculo al DataFrame de Spark, usando applyInPandas (Spark 3.x)
em_edad = emigrantes.groupBy('NOMBRE_PRO').applyInPandas(age_diff_udf, schema=age_diff_schema)

In [ ]:
em_edad.sample(0.01).toPandas()

# 3. Pandas UDF: windows

Vamos a añadir ahora otro dato a cada registro de emigrantes: cuántos emigrantes han venido a la provincia desde su misma provincia de origen (es decir, el tamaño de su cohorte).

En este caso vamos a usar un [Pandas UDF sobre una ventana](https://spark.apache.org/docs/latest/api/python/user_guide/arrow_pandas.html#series-to-scalar). 

In [ ]:
@pandas_udf("int")
def count_udf(v: pd.Series) -> int:
    '''Devolvemos el tamaño de la serie recibida (equivale al tamaño del grupo)'''
    return len(v)

Definimos ahora la ventana, usando [ventanas SQL](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html) definidas mediante el [API de Spark DataFrames](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Window.html)

Aplicaremos un particionado por provincias, y aplicando la UDF a todas las filas de la partición (*nota: en Spark 3.1 solo se soportan las ventanas definidas sobre **toda** la partición*).

In [ ]:
from pyspark.sql import Window

w = Window.partitionBy('NOMBRE_PRO').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

Usando la UDF, creamos una nueva columna que incluya, para cada fila del DataFrame, el número de emigrantes de la provincia de donde es la persona de esa fila

Para ello aplicamos la UDF **sobre** la ventana que particiona el DataFrame por provincias. La UDF es de tipo agregación: recibe una serie y produce un escalar.

In [ ]:
em_prov1 = emigrantes.withColumn('em_prov', count_udf(emigrantes['NOMBRE_PRO']).over(w))

In [ ]:
r = em_prov1.sample(0.01).limit(10).toPandas()

In [ ]:
r[['NOMBRE_PRO', 'IDHUECO', 'NORDEN', 'em_prov']]

## 3.1 Método alternativo

Nota: la cuenta de número de emigrantes por provincia se puede hacer fácilmente con los operadores estándar de agregación de DataFrames, pero en este caso las cuentas se añadirían al nivel de grupo, **no** al nivel individual

In [ ]:
emcount_prov = emigrantes.groupBy('NOMBRE_PRO').count().withColumnRenamed('count', 'num_emig')

In [ ]:
emcount_prov.limit(10).toPandas()

In [ ]:
em_prov2 = emigrantes.join(emcount_prov, on='NOMBRE_PRO') 

In [ ]:
# Nota: esto es lo mismo que arriba (pero más complicado)
em_prov2 = emigrantes.join(emcount_prov, on=[emigrantes.NOMBRE_PRO == emcount_prov.NOMBRE_PRO])
em_prov2 = em_prov2.drop(emcount_prov.NOMBRE_PRO)

In [ ]:
em_prov2.printSchema()

In [ ]:
r = em_prov2.sample(0.01).limit(10).toPandas()
r[['NOMBRE_PRO', 'IDHUECO', 'NORDEN', 'num_emig']]